# scikit-learn 첫걸음

이 노트북은 scikit-learn과의 첫 만남을 위한 실습입니다. 머신러닝의 기본 개념과 scikit-learn API의 일관성을 배우게 됩니다.

## 학습 목표
- scikit-learn 설치 확인
- 기본 데이터셋 탐색
- 첫 번째 머신러닝 모델 훈련
- scikit-learn API 이해

## 1. scikit-learn 설치 확인

In [ ]:
# 필수 라이브러리 임포트
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 버전 확인
print(f"scikit-learn 버전: {sklearn.__version__}")
print(f"NumPy 버전: {np.__version__}")
print(f"pandas 버전: {pd.__version__}")

# scikit-learn 주요 모듈 임포트
from sklearn import datasets, model_selection, preprocessing, metrics
print("✅ 모든 모듈이 성공적으로 임포트되었습니다!")

## 2. 첫 번째 데이터셋 탐색: Iris 데이터셋

Iris 데이터셋은 머신러닝에서 가장 유명한 데이터셋 중 하나입니다. 세 가지 종류의 붓꽃에 대한 꽃잎과 꽃받침 측정값을 포함합니다.

In [ ]:
# Iris 데이터셋 로드
iris = datasets.load_iris()

# 데이터 정보 확인
print(f"데이터셋 특성: {iris.feature_names}")
print(f"타겟 클래스: {iris.target_names}")
print(f"데이터 형태: {iris.data.shape}")
print(f"타겟 형태: {iris.target.shape}")

In [ ]:
# pandas DataFrame으로 변환하여 탐색
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target
iris_df['species_name'] = iris_df['species'].map({
    0: 'setosa', 
    1: 'versicolor', 
    2: 'virginica'
})

# 처음 5개 샘플 확인
print("처음 5개 샘플:")
iris_df.head()

In [ ]:
# 기본 통계 정보
print("데이터셋 통계:")
iris_df.describe()

In [ ]:
# 클래스 분포 확인
print("클래스별 샘플 수:")
iris_df['species_name'].value_counts()

In [ ]:
# 데이터 시각화
plt.figure(figsize=(12, 6))

# 꽃잎 길이 vs 꽃잎 너비
plt.subplot(1, 2, 1)
sns.scatterplot(
    data=iris_df, 
    x='petal length (cm)', 
    y='petal width (cm)', 
    hue='species_name', 
    style='species_name', 
    s=100
)
plt.title('꽃잎 길이 vs 꽃잎 너비')

# 꽃받침 길이 vs 꽃받침 너비
plt.subplot(1, 2, 2)
sns.scatterplot(
    data=iris_df, 
    x='sepal length (cm)', 
    y='sepal width (cm)', 
    hue='species_name', 
    style='species_name', 
    s=100
)
plt.title('꽃받침 길이 vs 꽃받침 너비')

plt.tight_layout()
plt.show()

## 3. 첫 번째 머신러닝 모델: K-최근접 이웃 (K-Nearest Neighbors)

이제 간단한 분류 모델을 훈련해 보겠습니다. K-최근접 이웃은 가장 직관적인 분류 알고리즘 중 하나입니다.

In [ ]:
# 재현성을 위한 랜덤 시드 설정
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 특성과 타겟 분리
X = iris.data  # 특성
y = iris.target  # 타겟

print(f"특성 형태: {X.shape}")
print(f"타겟 형태: {y.shape}")

In [ ]:
# 데이터 분할 (훈련 세트와 테스트 세트)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,  # 20%를 테스트용으로 사용
    random_state=RANDOM_STATE,  # 재현성을 위한 랜덤 시드
    stratify=y  # 클래스 비율 유지
)

print(f"훈련 세트 크기: {X_train.shape[0]}")
print(f"테스트 세트 크기: {X_test.shape[0]}")
print(f"훈련 세트 클래스 분포: {np.bincount(y_train)}")
print(f"테스트 세트 클래스 분포: {np.bincount(y_test)}")

In [ ]:
# K-최근접 이웃 분류기 생성 및 훈련
from sklearn.neighbors import KNeighborsClassifier

# 모델 생성 (k=3)
knn = KNeighborsClassifier(n_neighbors=3)

# 모델 훈련
knn.fit(X_train, y_train)

print("✅ 모델 훈련 완료!")

In [ ]:
# 예측 수행
y_pred = knn.predict(X_test)

# 성능 평가
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f"모델 정확도: {accuracy:.3f}")

In [ ]:
# 상세 성능 평가
from sklearn.metrics import classification_report, confusion_matrix

print("분류 보고서:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))

In [ ]:
# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=iris.target_names,
    yticklabels=iris.target_names
)
plt.xlabel('예측된 클래스')
plt.ylabel('실제 클래스')
plt.title('혼동 행렬')
plt.show()

## 4. 새로운 데이터에 대한 예측

이제 훈련된 모델을 사용하여 새로운 꽃 샘플을 분류해 보겠습니다.

In [ ]:
# 새로운 꽃 샘플 (꽃받침 길이, 꽃받침 너비, 꽃잎 길이, 꽃잎 너비)
new_flowers = np.array([
    [5.1, 3.5, 1.4, 0.2],  # setosa처럼 보임
    [6.7, 3.0, 5.2, 2.3],  # virginica처럼 보임
    [5.9, 3.0, 4.2, 1.5]   # versicolor처럼 보임
])

# 예측 수행
predictions = knn.predict(new_flowers)
predicted_species = [iris.target_names[p] for p in predictions]

print("새로운 꽃에 대한 예측:")
for i, (flower, species) in enumerate(zip(new_flowers, predicted_species)):
    print(f"꽃 {i+1} {flower} -> {species}")

## 5. scikit-learn API 일관성 탐색

scikit-learn의 강점은 모든 알고리즘이 동일한 API를 따른다는 것입니다. 다른 분류 알고리즘을 사용해 보겠습니다.

In [ ]:
# 다른 분류 알고리즘 시도
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# 모델 초기화
models = {
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'Logistic Regression': LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=RANDOM_STATE),
    'SVM': SVC(random_state=RANDOM_STATE)
}

# 모든 모델 훈련 및 평가
print("모델 성능 비교:")
print("-" * 30)

for name, model in models.items():
    # 동일한 API: fit()으로 훈련
    model.fit(X_train, y_train)
    
    # 동일한 API: predict()로 예측
    y_pred = model.predict(X_test)
    
    # 성능 평가
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name}: {accuracy:.3f}")

## 6. 모범 사례: 데이터 전처리와 모델 파이프라인

실제 머신러닝 프로젝트에서는 데이터 전처리가 중요합니다. 스케일링이 모델 성능에 미치는 영향을 살펴보겠습니다.

In [ ]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

# 스케일러 생성 및 훈련 데이터에 맞춤
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터는 동일한 스케일러로 변환
X_test_scaled = scaler.transform(X_test)

print("스케일링 전 데이터 통계:")
print(f"평균: {X_train.mean(axis=0)}")
print(f"표준편차: {X_train.std(axis=0)}")

print("\n스케일링 후 데이터 통계:")
print(f"평균: {X_train_scaled.mean(axis=0)}")
print(f"표준편차: {X_train_scaled.std(axis=0)}")

In [ ]:
# 스케일링된 데이터로 모델 훈련 및 평가
print("스케일링된 데이터로 모델 성능 비교:")
print("-" * 40)

for name, model in models.items():
    # 스케일링된 데이터로 훈련
    model.fit(X_train_scaled, y_train)
    
    # 스케일링된 테스트 데이터로 예측
    y_pred = model.predict(X_test_scaled)
    
    # 성능 평가
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name}: {accuracy:.3f}")

## 7. 요약 및 다음 단계

이 노트북에서는 다음을 배웠습니다:

1. ✅ scikit-learn 설치 및 환경 설정
2. ✅ 기본 데이터셋 탐색 및 시각화
3. ✅ 데이터 분할 (훈련/테스트)
4. ✅ 첫 번째 머신러닝 모델 훈련
5. ✅ 모델 성능 평가
6. ✅ scikit-learn API의 일관성
7. ✅ 데이터 전처리의 중요성

### 다음 단계:
- 모듈 2: 데이터 전처리 및 특성 공학 심화
- 다양한 전처리 기법 학습
- 더 복잡한 데이터셋으로 연습

### 추가 연습:
- 다른 k 값으로 KNN 모델 훈련 및 성능 비교
- 다른 scikit-learn 데이터셋으로 실험
- 교차 검증에 대해 알아보기